In [ ]:
import pandas as pd
import os

In [ ]:
# Ruta del archivo
ruta_archivo = '/home/valenciag17/datasets/competencia_03_crudo.csv.gz'

# Leer el archivo especificando el separador
df = pd.read_csv(ruta_archivo)

In [ ]:
# Asegurarse de que las columnas están en el formato correcto
# Asegurar que foto_mes sea un entero
df['foto_mes'] = df['foto_mes'].astype(int)

In [ ]:
# Ordenar los datos por cliente y por periodo
df = df.sort_values(by=['numero_de_cliente', 'foto_mes']).reset_index(drop=True)

# Obtener el periodo más reciente
ultimo_periodo = df['foto_mes'].max()

# Crear columnas temporales para verificar la continuidad de registros
df['siguiente_mes'] = df.groupby('numero_de_cliente')['foto_mes'].shift(-1)
df['dos_meses_despues'] = df.groupby('numero_de_cliente')['foto_mes'].shift(-2)

# Condiciones para asignar clase_ternaria
df['clase_ternaria'] = 'CONTINUA'  # Por defecto

# Identificar BAJA+1: cuando no existe el siguiente mes
df.loc[(df['siguiente_mes'].isna()) & (df['foto_mes'] < ultimo_periodo), 'clase_ternaria'] = 'BAJA+1'

# Identificar BAJA+2: cuando no existe dos meses después y el cliente no es BAJA+1
df.loc[
    (df['dos_meses_despues'].isna()) &
    (df['foto_mes'] < ultimo_periodo - 1) &
    (df['clase_ternaria'] != 'BAJA+1'),
    'clase_ternaria'
] = 'BAJA+2'

# Eliminar columnas temporales
df.drop(columns=['siguiente_mes', 'dos_meses_despues'], inplace=True)

# Mostrar resultados para verificar
print(df.head(20))

In [ ]:
# Si es necesario guardar los resultados
df.to_csv('/home/valenciag17/buckets/b1/datasets/competencia_03_final.csv.gz', index=False)